In [1]:
!pip install mlflow keras-tuner dvc -q
!pip install dvc[s3]
!apt install git -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.

In [2]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.keras
import os
import subprocess
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

In [ ]:
MLFLOW_TRACKING_URI = ""
EXPERIMENT_NAME = "AQI hyperparameter & model testing (Manual)"
WINDOW_SIZE = 24
PREDICT_HORIZON = 72  # Predict next 3 days = 72 hours

In [4]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/umair/mlruns/3', creation_time=1752022350617, experiment_id='3', last_update_time=1752022350617, lifecycle_stage='active', name='AQI hyperparameter & model testing (Manual)', tags={}>

In [5]:
!git init
!dvc init
!dvc remote add -d myremote s3://s3-bucket-umairrr

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                  

In [6]:
!git clone https://github.com/uma1r111/10pearls-AQI-Project-
%cd 10pearls-AQI-Project-

Cloning into '10pearls-AQI-Project-'...
remote: Enumerating objects: 404, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 404 (delta 80), reused 145 (delta 49), pack-reused 224 (from 1)
Receiving objects: 100% (404/404), 5.68 MiB | 20.48 MiB/s, done.
Resolving deltas: 100% (188/188), done.
/content/10pearls-AQI-Project-


In [ ]:
# Load and preprocess data
print("\nPulling latest feature_selection.csv from DVC remote (S3)...")
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = ""
subprocess.run(["dvc", "pull"], check=True)


Pulling latest feature_selection.csv from DVC remote (S3)...


CompletedProcess(args=['dvc', 'pull'], returncode=0)

In [8]:
for root, dirs, files in os.walk(".", topdown=True):
    for name in files:
        print(os.path.join(root, name))

./README.md
./.dvcignore
./requirements.txt
./.gitignore
./full_preprocessed_aqi_weather_data_with_all_features.csv
./karachi_weather_apr1_to_current.csv
./fetch_daily_data.py
./feature_selection.csv.dvc
./feature_selection.csv
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.ipynb
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.txt
./.dvc/config
./.dvc/.gitignore
./.dvc/cache/files/md5/a3/42be088d6bbebcb8002a166df6f909
./.dvc/tmp/btime
./.dvc/tmp/lock
./Data Preprocessing/Data_PreProcessing.ipynb
./Data Preprocessing/data_quality_check.py
./Data Preprocessing/run_preprocessing.py
./Data Collection/karachi_weather_jun2025.ipynb
./Data Collection/pollutants info Apr - Jun.ipynb
./Data Collection/karachi_weather_apr1_jun14.ipynb
./.github/workflows/feature_engineering.yml
./.github/workflows/feature_selection.yml
./.github/workflows/update_data.yml
./Feature Selection/initial_feature_extraction.ipynb
./Feature Selection/feature_selection.py


In [9]:
df = pd.read_csv("feature_selection.csv")

In [10]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df[(df["datetime"] >= "2025-04-01") & (df["datetime"] <= "2025-07-07")]
df = df.sort_values("datetime")

features = df.drop(columns=["datetime"])
num_features = features.shape[1]  # total features including AQI

In [11]:
print(df["datetime"].head())
print("Filtered Rows:", len(df))

0   2025-04-01 00:00:00
1   2025-04-01 01:00:00
2   2025-04-01 02:00:00
3   2025-04-01 03:00:00
4   2025-04-01 04:00:00
Name: datetime, dtype: datetime64[ns]
Filtered Rows: 2329


# Sequence Building for Walk forward **Validation**

In [12]:
def create_sequences(X, window_size, horizon):
    Xs, ys = [], []
    for i in range(len(X) - window_size - horizon):
        Xs.append(X[i:(i + window_size)].values)
        ys.append(X[(i + window_size):(i + window_size + horizon)].values)
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(features, WINDOW_SIZE, PREDICT_HORIZON)
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [13]:
print("Total Samples:", len(features))
print("X_seq shape:", X_seq.shape)
print("y_seq shape:", y_seq.shape)

Total Samples: 2329
X_seq shape: (2233, 24, 13)
y_seq shape: (2233, 72, 13)


# **Keras Hyperparameter Tuning**

In [16]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('n_units', min_value=32, max_value=128, step=32),
                   return_sequences=hp.Int('n_lstm_layers', min_value=1, max_value=2, step=1) > 1,
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   activation=hp.Choice('activation', values=['relu', 'tanh']),
                   dropout=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1),
                   recurrent_dropout=hp.Float('recurrent_dropout', min_value=0.0, max_value=0.3, step=0.1)))

    if hp.Int('n_lstm_layers', min_value=1, max_value=2, step=1) > 1:
        model.add(LSTM(units=hp.Int('n_units', min_value=32, max_value=128, step=32),
                       activation=hp.Choice('activation', values=['relu', 'tanh']),
                       dropout=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1),
                       recurrent_dropout=hp.Float('recurrent_dropout', min_value=0.0, max_value=0.3, step=0.1)))

    model.add(Dense(units=hp.Int('n_dense_units', min_value=32, max_value=128, step=32),
                    activation=hp.Choice('activation', values=['relu', 'tanh'])))
    model.add(Dense(PREDICT_HORIZON * X_train.shape[2]))

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')),
                  loss='mse')
    return model

# Run tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='aqi_multifeature_lstm'
)

with mlflow.start_run(run_name="Keras_Tuner_MultiOutput_Best_Hyperparameters"):
    tuner.search(X_train, y_train.reshape(y_train.shape[0], -1),
                 epochs=50,
                 batch_size=32,
                 validation_data=(X_val, y_val.reshape(y_val.shape[0], -1)),
                 callbacks=[EarlyStopping(monitor='val_loss', patience=5)],
                 verbose=0)

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    mlflow.log_params(best_hps.values)

    best_model = tuner.get_best_models(num_models=1)[0]
    val_preds = best_model.predict(X_val).reshape(y_val.shape)

    rmse = sqrt(mean_squared_error(y_val.flatten(), val_preds.flatten()))
    mlflow.log_metric("best_rmse", rmse)

    print("Best hyperparameters:", best_hps.values)
    mlflow.log_metric("best_rmse", rmse)


Reloading Tuner from tuner_dir/aqi_multifeature_lstm/tuner0.json


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a l

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
Best hyperparameters: {'n_units': 128, 'n_lstm_layers': 1, 'activation': 'tanh', 'dropout': 0.5, 'recurrent_dropout': 0.1, 'n_dense_units': 128, 'learning_rate': 0.004902441025672476}
🏃 View run Keras_Tuner_MultiOutput_Best_Hyperparameters at: http://172.174.154.85:8000/#/experiments/3/runs/0bb79999b14a4a9f88d63759eb6780a6
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **LSTM Model Fit**

In [18]:
with mlflow.start_run(run_name="Final_LSTM_MultiOutput_Keras"):
    model = Sequential()
    model.add(LSTM(units=best_hps.get('n_units'),
                   return_sequences=(best_hps.get('n_lstm_layers') > 1),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   activation=best_hps.get('activation'),
                   dropout=best_hps.get('dropout'),
                   recurrent_dropout=best_hps.get('recurrent_dropout')))

    if best_hps.get('n_lstm_layers') > 1:
        model.add(LSTM(units=best_hps.get('n_units'),
                       activation=best_hps.get('activation'),
                       dropout=best_hps.get('dropout'),
                       recurrent_dropout=best_hps.get('recurrent_dropout')))

    model.add(Dense(units=best_hps.get('n_dense_units'),
                    activation=best_hps.get('activation')))
    model.add(Dense(PREDICT_HORIZON * num_features))

    model.compile(optimizer=Adam(learning_rate=best_hps.get('learning_rate')),
                  loss='mse')

    model.fit(
        X_train, y_train.reshape(y_train.shape[0], -1),
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val.reshape(y_val.shape[0], -1)),
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )

    preds = model.predict(X_val).reshape(y_val.shape)
    rmse = sqrt(mean_squared_error(y_val.flatten(), preds.flatten()))
    mae = mean_absolute_error(y_val.flatten(), preds.flatten())

    mlflow.log_params(best_hps.values)
    mlflow.log_metric("final_rmse", rmse)
    mlflow.log_metric("final_mae", mae)
    mlflow.set_tag("model_type", "LSTM_MultiOutput")
    mlflow.set_tag("tuner", "Keras_Tuner")
    mlflow.set_tag("train_start", "2025-04-01")
    mlflow.set_tag("train_end", "2025-07-07")
    mlflow.keras.log_model(model, artifact_path="model")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - loss: 1653.7329 - val_loss: 671.5323
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 531.2009 - val_loss: 181.5218
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 186.7218 - val_loss: 72.0993
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 137.8881 - val_loss: 54.1161
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 120.2620 - val_loss: 51.4641
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 122.3063 - val_loss: 50.8463
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 122.4561 - val_loss: 50.8141
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 127.4527 - val_loss: 50.7801
Epoch 9/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 131.2006 - val_loss: 50.8377
Epoch 10/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 124.1547 - val_loss: 51.2053
Epoch 11/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 121.9670 - val_loss: 50.5940
Epoch 12/50
56/56 ━━━━━━━━━━━━━━━━━━━━

2025/07/09 22:56:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/09 22:56:25 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/07/09 22:56:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Final_LSTM_MultiOutput_Keras at: http://172.174.154.85:8000/#/experiments/3/runs/70a3808716594b0bb548840e684f0791
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **Predictions**

In [19]:
# Forecast next 3 days (72 hours)
last_sequence = features.values[-WINDOW_SIZE:]
current_input = last_sequence.reshape((1, WINDOW_SIZE, num_features))
next_features = model.predict(current_input).reshape(PREDICT_HORIZON, num_features)

future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
                              periods=PREDICT_HORIZON, freq='H')

future_df = pd.DataFrame(next_features, columns=features.columns)
future_df.insert(0, 'datetime', future_dates)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


/tmp/ipython-input-19-2459699523.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),


In [20]:
future_df.to_csv("future_multivariate_predictions.csv", index=False)
print("Future multivariate predictions saved to CSV.")
print(future_df.head())

Future multivariate predictions saved to CSV.
             datetime  aqi_us_lag1  aqi_us_lag12  aqi_us_lag24      pm2_5  \
0 2025-07-07 01:00:00    77.761421     77.835060     77.554214  23.677582   
1 2025-07-07 02:00:00    77.771004     77.826889     77.616440  23.692888   
2 2025-07-07 03:00:00    77.775375     77.781113     77.666122  23.777485   
3 2025-07-07 04:00:00    77.835457     77.759521     77.659943  23.968164   
4 2025-07-07 05:00:00    77.904274     77.760223     77.699493  23.931942   

   log_pm10  scaled_humidity_%  scaled_temp_C_scaled_log_windspeed_kph  \
0  4.104764          -0.363212                                0.396312   
1  4.125227          -0.408150                                0.421878   
2  4.136569          -0.481940                                0.690020   
3  4.129241          -0.440905                                0.190190   
4  4.177013          -0.414492                                0.340902   

    log_so2  day_of_week  scaled_temp_C  scale